# Example of using Llamaindex framework for Retrieval Augmented Generation
This notebook shows how to run Llamaindex framework locally to create virtual AI assistant based on RAG (Retrieval Augmented Generation).
For dataset to search for source information wikipedia articles about cryptocurrencies were used.

## Llamaindex setup
### Download llamafile
Download the llamafile with model. Llamafile can contain any LLM. Framework enabled to run it as a local server and use via API. 
TinyLlama-1.1B-Chat-v1.0 model is used for purpose of this example

` wget https://huggingface.co/Mozilla/TinyLlama-1.1B-Chat-v1.0-llamafile/resolve/main/TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile`

Make executable 

`chmod +x TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile`

Run in server mode

`./TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile --server --nobrowser --embedding --port 8081`

### Install Llamaindex Python library

In [1]:
# Install llama-index
!pip install llama-index 
# Install llamafile integrations and SimpleWebPageReader
!pip install llama-index-embeddings-llamafile llama-index-llms-llamafile llama-index-readers-web

  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.3 MB/s eta 0:00:00
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.20.1
    Uninstalling pydantic_core-2.20.1:
      Successfully uninstalled pydantic_core-2.20.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.19
    Uninstalling pydantic-1.10.19:
      Successfully uninstalled pydantic-1.10.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
fastapi 0.

## Configuration

In [34]:
# Configure LlamaIndex
from llama_index.core import Settings
from llama_index.embeddings.llamafile import LlamafileEmbedding
from llama_index.llms.llamafile import Llamafile
from llama_index.core.node_parser import SentenceSplitter

#configure object to encode text into vector using started endpoint
Settings.embed_model = LlamafileEmbedding(base_url="http://localhost:8081")

#configure object that will use model endpoint
Settings.llm = Llamafile(
    base_url="http://localhost:8081",
    temperature=0,
    seed=0
)

#configure split text to chunks 
Settings.transformations = [
    SentenceSplitter(
        chunk_size=256, 
        chunk_overlap=5
    )
]

In [35]:
# Load local data with some cryptocurrencies descriptions
from llama_index.core import SimpleDirectoryReader
# create reader for local documents
local_doc_reader = SimpleDirectoryReader(input_dir='./data/cryptocurrency_wikipedia')
# create collection that will contain all documents used for retrieval
docs = local_doc_reader.load_data(show_progress=True)














Loading files: 100%|██████████| 3/3 [00:00<00:00, 2289.88file/s][A


In [36]:
# Add Wikipedia pages
from llama_index.readers.web import SimpleWebPageReader
urls = [
    'https://en.wikipedia.org/wiki/Bitcoin',
    'https://en.wikipedia.org/wiki/Ethereum',
    'https://en.wikipedia.org/wiki/Dogecoin'
]
# create reader that can fetch websites content
web_reader = SimpleWebPageReader(html_to_text=True)
# add fetched content to docs collection
docs.extend(web_reader.load_data(urls))

In [37]:
from llama_index.core import VectorStoreIndex

# create index storage with embedded documents
index = VectorStoreIndex.from_documents(
    docs,
    show_progress=True,
)

# dump storage locally
index.storage_context.persist(persist_dir="./storage")

Parsing nodes:   0%|          | 0/6 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/547 [00:00<?, ?it/s]

HTTPStatusError: Server error '503 Service Unavailable' for url 'http://localhost:8081/embedding'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/503

In [38]:
# create query type engine to ask questions to llm providing documents from index
query_engine = index.as_query_engine()
# ask questions about data from storage
print(query_engine.query("What is Coinye?"))

NameError: name 'index' is not defined

In [39]:
print(query_engine.query("Is Dogecoin stable?"))

NameError: name 'query_engine' is not defined

In [28]:
print(query_engine.query("Is Bitcoin good for environment?"))

Yes, Bitcoin is considered to be a good option for environment as it does not require any physical infrastructure, such as mining facilities, to operate. It is a decentralized and secure cryptocurrency that uses blockchain technology to verify transactions and maintain a decentralized ledger. This means that Bitcoin does not require any intermediaries or third-party entities to process transactions, which reduces the carbon footprint associated with traditional financial systems. Additionally, Bitcoin's energy consumption is significantly lower than that of traditional financial systems, making it a more sustainable option for the environment.</s>
